In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
import urllib.request as request
from urllib.parse import quote

import ruptures as rpt

from datetime import datetime
from datetime import timedelta
from tqdm import tqdm
import pyodbc 

pd.options.mode.chained_assignment = None

In [4]:
df = pd.read_excel(r'C:\Users\misc\Downloads\MeteringPoints.xlsx')

In [5]:
df.head()

,MålepunktsID,MålepunktsID_hovedmåler,Alias,Målepunktstype,Netområde,Nettoafregningsgruppe,Tilslutningsstatus,Branchekode,Effektgrænse_kW,Effektgrænse_ampere,...,Juridisk_kontakt_Husnr.,Juridisk_kontakt_Etage,Juridisk_kontakt_Dør,Juridisk_kontakt_Postnr.,Juridisk_kontakt_By,Juridisk_kontakt_Stednavn,Juridisk_kontakt_Land,Juridisk_kontakt_Telefonnr.,Juridisk_kontakt_Mobilnr.,Juridisk_kontakt_E-mail
0,571313174110568005,NaN,NaN,Forbrugsmålepunkt,791,0.0,Tilsluttet,433.0,0.0,0.0,...,7.0,NaN,NaN,2750.0,Ballerup,NaN,DK,NaN,44776191.0,bsh@kab-bolig.dk
1,571313174110568012,NaN,NaN,Forbrugsmålepunkt,791,0.0,Tilsluttet,433.0,0.0,0.0,...,7.0,NaN,NaN,2750.0,Ballerup,NaN,DK,NaN,44776191.0,bsh@kab-bolig.dk
2,571313174110568043,NaN,NaN,Forbrugsmålepunkt,791,0.0,Tilsluttet,445.0,0.0,0.0,...,7.0,NaN,NaN,2750.0,Ballerup,NaN,DK,NaN,44776191.0,bsh@kab-bolig.dk
3,571313174110568050,NaN,NaN,Forbrugsmålepunkt,791,0.0,Tilsluttet,445.0,0.0,0.0,...,7.0,NaN,NaN,2750.0,Ballerup,NaN,DK,NaN,44776191.0,bsh@kab-bolig.dk
4,571313174110568074,NaN,NaN,Forbrugsmålepunkt,791,0.0,Tilsluttet,433.0,0.0,0.0,...,7.0,NaN,NaN,2750.0,Ballerup,NaN,DK,NaN,44776191.0,bsh@kab-bolig.dk


In [13]:
df['adresse'] = df['Vejnavn'].astype(str) + ' ' + df['Husnummer'].astype(str) + ', ' + df['Postnummer'].astype(str) + ' ' + df['By'].astype(str)

In [14]:
df.head()

,MålepunktsID,MålepunktsID_hovedmåler,Alias,Målepunktstype,Netområde,Nettoafregningsgruppe,Tilslutningsstatus,Branchekode,Effektgrænse_kW,Effektgrænse_ampere,...,Juridisk_kontakt_Etage,Juridisk_kontakt_Dør,Juridisk_kontakt_Postnr.,Juridisk_kontakt_By,Juridisk_kontakt_Stednavn,Juridisk_kontakt_Land,Juridisk_kontakt_Telefonnr.,Juridisk_kontakt_Mobilnr.,Juridisk_kontakt_E-mail,adresse
0,571313174110568005,NaN,NaN,Forbrugsmålepunkt,791,0.0,Tilsluttet,433.0,0.0,0.0,...,NaN,NaN,2750.0,Ballerup,NaN,DK,NaN,44776191.0,bsh@kab-bolig.dk,"Gl Rådhusvej 13, 2750 Ballerup"
1,571313174110568012,NaN,NaN,Forbrugsmålepunkt,791,0.0,Tilsluttet,433.0,0.0,0.0,...,NaN,NaN,2750.0,Ballerup,NaN,DK,NaN,44776191.0,bsh@kab-bolig.dk,"Gl Rådhusvej 13, 2750 Ballerup"
2,571313174110568043,NaN,NaN,Forbrugsmålepunkt,791,0.0,Tilsluttet,445.0,0.0,0.0,...,NaN,NaN,2750.0,Ballerup,NaN,DK,NaN,44776191.0,bsh@kab-bolig.dk,"Solvej 6, 2750 Ballerup"
3,571313174110568050,NaN,NaN,Forbrugsmålepunkt,791,0.0,Tilsluttet,445.0,0.0,0.0,...,NaN,NaN,2750.0,Ballerup,NaN,DK,NaN,44776191.0,bsh@kab-bolig.dk,"Torvevej 19, 2740 Skovlunde"
4,571313174110568074,NaN,NaN,Forbrugsmålepunkt,791,0.0,Tilsluttet,433.0,0.0,0.0,...,NaN,NaN,2750.0,Ballerup,NaN,DK,NaN,44776191.0,bsh@kab-bolig.dk,"Rugvænget 10, 2750 Ballerup"


In [18]:
df['adresse'].nunique()

327

In [19]:
df['Anslået_årsforbrug'].sum()

14227708

In [20]:
df.shape

(439, 73)

In [21]:
df['MålepunktsID'].nunique()

439